In [1]:
import os
import pandas as pd

In [2]:
wd = os.getcwd()
wgbstools = '/sc/projects/sci-kretzmer/chair/software/tools/wgbs_tools/wgbstools'

In [3]:
def beta_simulate(bgab, abab):
    try:
        os.mkdir(wd+'/data')
    except:
        pass
    
    try:
        os.mkdir(wd+'/data/'+bgab.replace(' ',''))
    except:
        pass
    
    mxqsh = wd+"/data/bg_"+bgab.replace(' ','')+"_mxq.sh"
    
    with open(mxqsh,'w') as fn:
        for i in range(50):
            fn.write('Rscript '+wd+'/simulate_background.R '+bgab+' '+\
                        wd+'/chromatin_annotation_chr10.txt '+str(i)+' > '+\
                        wd+'/data/'+bgab.replace(' ','')+'/sample_'+str(i)+'.txt \n')
    
    os.system('chmod 700 '+mxqsh+'; bash '+mxqsh)

In [4]:
def dmr_simulate(bgab, abab):
    os.system('Rscript '+wd+'/simulate_DMRs_WGBS_multi.R '+\
                        abab+' '+wd+'/data/'+bgab.replace(' ','')+' '+wd+'/data/'+bgab.replace(' ','')+' ')
    
    met = pd.read_table(wd+'/data/'+bgab.replace(' ','')+'/metilene.beta_'+abab.replace(' ','_')+'.tsv')
    met['End'] = met['pos']+1
    met = met[['chr','pos','End']+sorted(met.columns[met.columns.str.contains('_')])]
    newids = []
    j = 0
    g = '0'
    for i in sorted(met.columns[met.columns.str.contains('_')]):
        if i.split('_')[0]==g:
            j+=1
        else:
            g = i.split('_')[0]
            j=1
        newids.append('G'+i.split('_')[0]+'_'+str(j))
    
    met.columns = ['Chrome','Start','End']+newids
    met.to_csv(wd+'/data/'+bgab.replace(' ','')+'/smart.beta_'+abab.replace(' ','_')+'.tsv', sep='\t', index=False)
    
    indir = wd+'/data/'+bgab.replace(' ','')
    s = pd.DataFrame(os.listdir(indir))
    s = s.loc[s[0].str.contains('wgbstools.bed')]
    s[0] = s[0].apply(lambda x:x.replace('.bed',''))
    s[1] = s[0].apply(lambda x:'G'+x.split('_')[0])
    s.columns = ['name','group']
    s.to_csv(indir+'/groups.wgbstools.csv',index=False)
    
    os.system('cd '+indir+';'+wgbstools+' bed2beta '+indir+'/*.wgbstools.bed')
    
    s = pd.DataFrame(os.listdir(indir))
    s = s.loc[s[0].str.contains('wgbstools.bed')]
    s[0] = s[0].apply(lambda x:x.split('.')[0])
    s[1] = s[0].apply(lambda x:'G'+x.split('_')[0])
    s.columns = ['ID','Group']
    s.to_csv(indir+'/groups.metilene.tsv',index=False, sep='\t')
    
    return pd.crosstab(pd.read_table(wd+'/data/'+bgab.replace(' ','')+'/DMR_nooverlapped_beta_'+abab.replace(' ','_')+'.bed',header=None)[6].apply(lambda x:x[:5]),
    pd.read_table(wd+'/data/'+bgab.replace(' ','')+'/DMR_nooverlapped_beta_'+abab.replace(' ','_')+'.bed',header=None)[7], margin=True)

In [ ]:
bgab = '40 3'
abab = bgab+' 22 22'
beta_simulate(bgab, abab)

In [ ]:
bgab = '15 5'
abab = bgab+' 10 10'
beta_simulate(bgab, abab)

In [ ]:
bgab = '40 3'
abab = bgab+' 22 22'
dmr_simulate(bgab, abab)

In [ ]:
bgab = '15 5'
abab = bgab+' 10 10'
dmr_simulate(bgab, abab)